## Creating schools.csv

1. Install packages
2. Create cities.csv with full state name/ city column to use in getting school information
3. For persisitance creating a schools csv using selenium to get school information from greatschools.org
4. Clean csv for use in schools endpoint

### 1. Import necessary libraries

In [1]:
# import os
# import requests
# import csv
# import json
# import pandas as pd
# from state_abbr import us_state_abbrev as abbr
# from selenium import webdriver
# import urllib.parse

### 2.  Create cities.csv with full state name/ city column to use in getting school information

In [2]:
# pwd = os.getcwd()

# # create city state list
# cities = pd.read_excel('notebooks/datasets/data/schools/csv/List of Cities.xlsx')

# # just get the second and third colun
# cities = cities[['Unnamed: 1','Unnamed: 2']]

# # create new dictionary with reversed key, value pairs
# full = dict(map(reversed, abbr.items()))

# # map state abbreviations to full name
# cities['states'] = cities['Unnamed: 2'].map(full)

# # making sure state/city combo conform to url format of "-" for " "
# cities['states'] = cities['states'].str.strip()
# cities['states'] = cities['states'].str.replace(" ", "-")
# cities['Unnamed: 1'] = cities['Unnamed: 1'].str.replace(" ", "-")

# # remove extraneous header rows
# cities = cities.iloc[2:]
# cities['city'] = (cities['states'] + '/'+ cities['Unnamed: 1']).str.lower()
# print(cities.head())

# # persist by creating new csv
# cities.to_csv('notebooks/datasets/data/schools/csv/cities.csv')

### 3. For persisitance creating a schools csv using selenium to get school information from greatschools.org

In [3]:
# # using selenium to get school information
# driver = webdriver.Chrome()

In [4]:
# # url for greatschools pre_url and post_url (with state/city inbetween)
# url_pre = 'http://www.greatschools.org/'
# url_post = '/schools/?tableView=Overview&view=table'

# # Call cities csv to get cities stored in database
# cities = pd.read_csv('notebooks/datasets/data/schools/csv/cities.csv')

In [5]:
# # Looping through each city in the file

# # create empty dataframe
# df = pd.DataFrame()

# for i in cities['city']:
#     endpoint = url_pre + urllib.parse.quote(i) + url_post
#     print("Fetching ", endpoint)
#     driver.get(endpoint)
#     html = driver.page_source
#     table = pd.read_html(html)
#     # appending to dataframe all the schhol information for current cities
#     df = df.append(table[0])

# driver.close()

In [6]:
# # 5. For persisitance creating a schools csv
# df.to_csv('notebooks/datasets/data/schools/csv/schools.csv')

### 4a. Clean csv for use in schools endpoint

In [7]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)

/Users/jisha/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [8]:
df = pd.read_csv('csv/schools.csv')

In [9]:
# Drop unused columns
df = df.drop(columns=['Unnamed: 0', 'Reviews'])

In [10]:
# Extract numerical rating
df[['Score', 'School']] = df['School'].str.split('/10', n=2, expand=True)

In [11]:
# Extract score 
# https://stackoverflow.com/questions/4416425/how-to-split-string-with-some-separator-but-without-removing-that-separator-in-j
df['Rating'] = df['School'].str.split('(?<=[a-z])(?=[A-Z])')
df['Rating'] = df['Rating'].str[0]

In [12]:
# Cleaning strings

# School
df['School'] = df['School'].str.replace('Top rated', '')
df['School'] = df['School'].str.replace('Above average', '')
df['School'] = df['School'].str.replace('Average', '')
df['School'] = df['School'].str.replace('Below average', '')
df['School'] = df['School'].str.replace('None', '')
df['School'] = df['School'].str.replace(' Homes for sale', '')
df['School'] = df['School'].str.replace('2 awardsAwards & BadgesCollege Success Award2019, 2020See all winners in ', '')
df['School'] = df['School'].str.replace('3 awardsAwards & BadgesCollege Success Award2019, 2020See all winners in ', '')

# Score
df['Rating'] = df['Rating'].str.replace('Below average7th & 8th Grade Center402 North S Street, Muskogee, OK, 74403 Homes for sale', 'Below average')
df['Rating'] = df['Rating'].str.replace('Below average7th & 8th Grade Center402 North S Street, Muskogee, OK, 74403 Homes for sale', 'Below average')
df['Rating'] = df['Rating'].str.replace('Average12th Street Elementary6501 South 12th Street, Portage, MI, 49024 Homes for sale', 'Average')
df['Rating'] = df['Rating'].str.replace('Below average21st Century Preparatory School1220 Mound Avenue, Racine, WI, 53404 Homes for sale', 'Below average')
df['Rating'] = df['Rating'].str.replace('Below averagee', 'Below average')
df['Rating'] = df['Rating'].str.replace('Averagedu', 'Average')


In [13]:
df['Address'] = df['School'].str.split('(?<=[a-z])(?=[0-9])')

In [14]:
# Finding length because there are anomalies with the information in the address column
df['Length'] = df['Address'].apply(lambda x: len(x) if x != None else 0 )
# 2 is the expected length -> school and address
df['Length'].unique()

array([4, 2, 0, 1, 3])

In [15]:
# School Name 
df['School'] = df['Address'].str[0]

In [16]:
# This where the address should be in most cases except the ones above
df['Address'] = df['Address'].str[-1]

In [17]:
print(df.shape)
df.head()

(9741, 10)


,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
0,Akron Early College High School,Public district,9-12,384.0,34:1,Akron City School District,10,Top rated,"225 South Main Street, Akron, OH, 44308",4
1,Revere Middle School,Public district,6-8,624.0,13:1,Revere Local School District,9,Above average,"3195 Spring Valley Road, Akron, OH, 44333",2
2,Arrowhead Primary Elementary School,Public district,K-4,345.0,20:1,Copley-Fairlawn City School District,8,Above average,"1600 Raleigh Boulevard, Akron, OH, 44321",2
3,Manchester Middle School,Public district,5-8,387.0,16:1,Manchester Local School District,8,Above average,"760 West Nimisila Road, Akron, OH, 44319",2
4,Nolley Elementary School,Public district,K-4,483.0,17:1,Manchester Local School District,8,Above average,"6285 Renninger Rd, Akron, OH, 44319",2


### 4b. Further cleaning

In [18]:
# These are the rows that need extra tlc
unrated = df.loc[df['Length'] == 0]
single = df.loc[df['Length'] == 1]
miscellaneous = df.loc[df['Length'] == 3]

In [19]:
# Removing rows where the length is not 2 or 4
df = df[df['Length'] != 0]
df = df[df['Length'] != 1]
df = df[df['Length'] != 3]

### Unrated

In [20]:
print(unrated.shape)
unrated.head()

(1988, 10)


,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
46,None,Private,PK-8,110.0,NaN,NaN,"Currently unratedSt. Teresa's School417 Edgewood Lane, Albany, GA, 31707 Homes for sale",None,None,0
47,None,Private,PK-12,727.0,NaN,NaN,"Currently unratedDeerfield-Windsor SchoolPo Box 71149, Albany, GA, 31708 Homes for sale",None,None,0
48,None,Private,9-12,NaN,NaN,NaN,"Currently unratedDarsey Private School1201 W Oglethorpe Ave, Albany, GA, 31707 Homes for sale",None,None,0
49,None,Private,7-11,NaN,NaN,NaN,"Currently unratedNew Covenant Academy1510 W Broad Avenue, Albany, GA, 31707 Homes for sale",None,None,0
92,None,Private,PK-8,95.0,NaN,NaN,"Currently unratedSt Mary Elementary School815 Broadalbin Street Southwest, Albany, OR, 97321 Homes for sale",None,None,0


In [21]:
unrated['School'] = unrated['Score']
unrated['Score'] = 0

In [22]:
unrated['Rating'] = unrated['School'].str.split('(?<=[a-z])(?=[A-Z])')

In [23]:
unrated['Rating'] = unrated['Rating'].str[0]

In [24]:
unrated['School'] = unrated['School'].str.replace(' Homes for sale', '')
unrated['School'] = unrated['School'].str.replace('Currently unrated', '')

In [25]:
unrated['Address'] = unrated['School'].str.split('(?<=[a-z])(?=[0-9])')

In [26]:
# Finding length because there are anomalies with the information in the address column
unrated['Length'] = unrated['Address'].apply(lambda x: len(x) if x != None else 0 )
# 2 is the expected length -> school and address
unrated['Length'].unique()

array([2, 1, 3])

In [27]:
unrated['School'] = unrated['Address'].str[0]

In [28]:
unrated['Address'] = unrated['Address'].str[-1]

In [29]:
# These will also need further cleaning
unrated_1 = unrated.loc[unrated['Length']==1]
unrated_3 = unrated.loc[unrated['Length']==3]

In [30]:
# Removing rows where the length is not 2
unrated = unrated[unrated['Length'] != 1]
unrated = unrated[unrated['Length'] != 3]

In [31]:
unrated[['Total students enrolled', 'Students per teacher', 'District' ]] = unrated[['Total students enrolled', 'Students per teacher', 'District' ]].fillna(0)

In [32]:
# Mergeable unrated df
print(unrated.shape)
unrated.head()

(1829, 10)


,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
46,St. Teresa's School,Private,PK-8,110.0,0,0,0,Currently unrated,"417 Edgewood Lane, Albany, GA, 31707",2
48,Darsey Private School,Private,9-12,0.0,0,0,0,Currently unrated,"1201 W Oglethorpe Ave, Albany, GA, 31707",2
49,New Covenant Academy,Private,7-11,0.0,0,0,0,Currently unrated,"1510 W Broad Avenue, Albany, GA, 31707",2
92,St Mary Elementary School,Private,PK-8,95.0,0,0,0,Currently unrated,"815 Broadalbin Street Southwest, Albany, OR, 97321",2
93,Albany Christian School,Private,PK-8,261.0,0,0,0,Currently unrated,"420 Southeast 3rd Avenue, Albany, OR, 97321",2


### Single

In [33]:
print(single.shape)
single.head()

(163, 10)


,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
349,"Woodland IntermediateN9085 Coop Rd, Appleton, WI, 54915",Public district,5-6,362.0,18:1,Kimberly Area School District,10,Top rated,"Woodland IntermediateN9085 Coop Rd, Appleton, WI, 54915",1
353,"Woodland SchoolN9085 N Coop Rd, Appleton, WI, 54915",Public district,K-4,494.0,14:1,Kimberly Area School District,9,Above average,"Woodland SchoolN9085 N Coop Rd, Appleton, WI, 54915",1
358,"Freedom Elementary SchoolN3569 County Road E, Appleton, WI, 54913",Public district,PK-5,705.0,19:1,Freedom Area School District,7,Above average,"Freedom Elementary SchoolN3569 County Road E, Appleton, WI, 54913",1
360,"Sunrise Elementary SchoolN9363 Exploration Avenue, Appleton, WI, 54915",Public district,K-4,423.0,14:1,Kimberly Area School District,7,Above average,"Sunrise Elementary SchoolN9363 Exploration Avenue, Appleton, WI, 54915",1
514,"BRIGHTON AVENUE SCHOOL30 North Brighton Avenue, Atlantic City, NJ, 08401",Public district,PK-5,346.0,10:1,Atlantic City Public School District,3,Below average,"BRIGHTON AVENUE SCHOOL30 North Brighton Avenue, Atlantic City, NJ, 08401",1


In [34]:
single['Address'] = single['School'].str.split('(?<=[a-z])(?=[A-Z])')

In [35]:
# Finding length because there are anomalies with the information in the address column
single['Length'] = single['Address'].apply(lambda x: len(x) if x != None else 0 )
# 2 is the expected length -> school and address
single['Length'].unique()

array([2, 1, 3])

In [36]:
single['School'] = single['Address'].str[0]

In [37]:
single['Address'] = single['Address'].str[1]

In [38]:
single_1 = single.loc[single['Length']==1]
single_3 = single.loc[single['Length']==3]

In [39]:
single = single[single['Length']!=1]
single = single[single['Length']!=3]

In [40]:
single['School'] = single['School'].str.replace('William B Travis Acdmy/Vngrd For A3001 Mc', 'William B Travis Acdmy/Vngrd For')
single['Address'] = single['Address'].str.replace('Kinney Avenue, Dallas, TX, 75204', 'A3001 McKinney Avenue, Dallas, TX, 75204')

In [41]:
# Mergeable single df
print(single.shape)
single.head()

(14, 10)


,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
349,Woodland Intermediate,Public district,5-6,362.0,18:1,Kimberly Area School District,10,Top rated,"N9085 Coop Rd, Appleton, WI, 54915",2
353,Woodland School,Public district,K-4,494.0,14:1,Kimberly Area School District,9,Above average,"N9085 N Coop Rd, Appleton, WI, 54915",2
358,Freedom Elementary School,Public district,PK-5,705.0,19:1,Freedom Area School District,7,Above average,"N3569 County Road E, Appleton, WI, 54913",2
360,Sunrise Elementary School,Public district,K-4,423.0,14:1,Kimberly Area School District,7,Above average,"N9363 Exploration Avenue, Appleton, WI, 54915",2
2225,William B Travis Acdmy/Vngrd For,Public district,4-5,200.0,15:1,Dallas Independent School District,10,Top rated,"A3001 McKinney Avenue, Dallas, TX, 75204",2


### Miscellaneous

In [42]:
# Mergeable miscellaneous df
print(miscellaneous.shape)
miscellaneous.head()

(3, 10)


,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
1906,The Vanguard School (High)3 awardsAwards & BadgesCollege Success Award,Public charter,9-12,305.0,10:1,Cheyenne Mountain 12 School District,8,Above average,"1832 South Wahsatch Avenue, Colorado Springs, CO, 80905",3
2947,International Academy of Flint (K-12)1 awardAwards & BadgesCollege Success Award,Public charter,K-12,1030.0,21:1,International Academy Of Flint,4,Below average,"2820 Saginaw Street, Flint, MI, 48503",3
3625,Achievement First Hartford Academy Inc.1 awardAwards & BadgesCollege Success Award,Public charter,K-12,1149.0,11:1,Achievement First Hartford Ac,6,Average,"305 Greenfield Street, Hartford, CT, 06112",3


In [43]:
miscellaneous['School'] = miscellaneous['School'].str.replace('1 awardAwards & BadgesCollege Success Award', '')
miscellaneous['School'] = miscellaneous['School'].str.replace('3 awardsAwards & BadgesCollege Success Award', '')

In [44]:
print(miscellaneous.shape)
miscellaneous.head()

(3, 10)


,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
1906,The Vanguard School (High),Public charter,9-12,305.0,10:1,Cheyenne Mountain 12 School District,8,Above average,"1832 South Wahsatch Avenue, Colorado Springs, CO, 80905",3
2947,International Academy of Flint (K-12),Public charter,K-12,1030.0,21:1,International Academy Of Flint,4,Below average,"2820 Saginaw Street, Flint, MI, 48503",3
3625,Achievement First Hartford Academy Inc.,Public charter,K-12,1149.0,11:1,Achievement First Hartford Ac,6,Average,"305 Greenfield Street, Hartford, CT, 06112",3


### Unrated_1

In [45]:
print(unrated_1.shape)
unrated_1.head()

(158, 10)


,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
47,"Deerfield-Windsor SchoolPo Box 71149, Albany, GA, 31708",Private,PK-12,727.0,NaN,NaN,0,Currently unrated,"Deerfield-Windsor SchoolPo Box 71149, Albany, GA, 31708",1
146,"St Rita Child Development CenterPo Box 7056, Alexandria, LA, 71306",Private,PK,NaN,NaN,NaN,0,Currently unrated,"St Rita Child Development CenterPo Box 7056, Alexandria, LA, 71306",1
147,"Bright Beginnings Child Care CenterPo Box 12264, Alexandria, LA, 71315",Private,PK-K,NaN,NaN,NaN,0,Currently unrated,"Bright Beginnings Child Care CenterPo Box 12264, Alexandria, LA, 71315",1
192,"Lily Pond Child Care Center-Juniata GapJuniata Gap Road, Altoona, PA, 16601",Private,PK,NaN,NaN,NaN,0,Currently unrated,"Lily Pond Child Care Center-Juniata GapJuniata Gap Road, Altoona, PA, 16601",1
195,"Lily Pond Sch/Age ProgramSheraton Drive, Altoona, PA, 16601",Private,PK,NaN,NaN,NaN,0,Currently unrated,"Lily Pond Sch/Age ProgramSheraton Drive, Altoona, PA, 16601",1


In [46]:
unrated_1['Address'] = unrated_1['School'].str.split('(?<=[a-z])(?=[A-Z])')

In [47]:
# Finding length because there are anomalies with the information in the address column
unrated_1['Length'] = unrated_1['Address'].apply(lambda x: len(x) if x != None else 0 )
# 2 is the expected length -> school and address
unrated_1['Length'].unique()

array([2, 1])

In [48]:
unrated_single = unrated_1.loc[unrated_1['Length'] == 1]

In [49]:
unrated_1['School'] = unrated_1['Address'].str[0]

In [50]:
unrated_1['Address'] = unrated_1['Address'].str[1]

In [51]:
unrated_1 = unrated_1[unrated_1['Length']!=1]

In [52]:
unrated_1[['Total students enrolled', 'Students per teacher', 'District' ]] = unrated_1[['Total students enrolled', 'Students per teacher', 'District' ]].fillna(0)

In [53]:
# Mergeable unrated_1
print(unrated_1.shape)
unrated_1.head()

(112, 10)


,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
47,Deerfield-Windsor School,Private,PK-12,727.0,0,0,0,Currently unrated,"Po Box 71149, Albany, GA, 31708",2
146,St Rita Child Development Center,Private,PK,0.0,0,0,0,Currently unrated,"Po Box 7056, Alexandria, LA, 71306",2
147,Bright Beginnings Child Care Center,Private,PK-K,0.0,0,0,0,Currently unrated,"Po Box 12264, Alexandria, LA, 71315",2
192,Lily Pond Child Care Center-Juniata Gap,Private,PK,0.0,0,0,0,Currently unrated,"Juniata Gap Road, Altoona, PA, 16601",2
195,Lily Pond Sch/Age Program,Private,PK,0.0,0,0,0,Currently unrated,"Sheraton Drive, Altoona, PA, 16601",2


### Unrated_3

In [54]:
print(unrated_3.shape)
unrated_3.head()

(1, 10)


,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
5704,Joy Quigley Day Care,Private,PK,NaN,NaN,NaN,0,Currently unrated,"94, Muscatine, IA, 52761",3


In [55]:
unrated_3[['Total students enrolled', 'Students per teacher', 'Students per teacher']] = unrated_3[['Total students enrolled', 'Students per teacher', 'Students per teacher']].fillna(0)

In [56]:
print(unrated_3.shape)
unrated_3.head()

(1, 10)


,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
5704,Joy Quigley Day Care,Private,PK,0.0,0,NaN,0,Currently unrated,"94, Muscatine, IA, 52761",3


### Single_1

In [57]:
print(single_1.shape)
single_1.head(25)

(148, 10)


,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
514,"BRIGHTON AVENUE SCHOOL30 North Brighton Avenue, Atlantic City, NJ, 08401",Public district,PK-5,346.0,10:1,Atlantic City Public School District,3,Below average,NaN,1
560,"Aurora Quest K-8472 South Wheeling Street, Aurora, CO, 80012",Public district,K-8,599.0,21:1,Adams-Arapahoe 28j School District,9,Above average,NaN,1
561,"Aurora Frontier K-83200 South Jericho Way, Aurora, CO, 80013",Public district,PK-8,697.0,21:1,Adams-Arapahoe 28j School District,8,Above average,NaN,1
718,"Bay County PSA1110 State Street, Bay City, MI, 48706",Public charter,K-5,101.0,11:1,Bay County Psa,5,Average,NaN,1
735,"Beaumont Isd Early College High S3395 Harrison Avenue, Beaumont, TX, 77706",Public district,9-12,127.0,NaN,Beaumont Independent School District,7,Above average,NaN,1
885,"Elysian 7-86416 Elysian Road, Billings, MT, 59101",Public district,7-8,85.0,11:1,Elysian Elementary School District,7,Above average,NaN,1
897,"Will James 7-81200 30th Street West, Billings, MT, 59102",Public district,6-8,522.0,18:1,Billings Elementary School District,5,Average,NaN,1
899,"Elder Grove 7-81532 South 64 St West, Billings, MT, 59106",Public district,6-8,200.0,20:1,Elder Grove Elementary School District,5,Average,NaN,1
905,"Castle Rock 7-81441 Governors Boulevard, Billings, MT, 59105",Public district,6-8,582.0,16:1,Billings Elementary School District,4,Below average,NaN,1
1235,"Gs Mickelson Middle School - 021801 12th Street South, Brookings, SD, 57006",Public district,6-8,796.0,15:1,Brookings School District 05-1,6,Average,NaN,1


In [58]:
# Too varied - need to do manual 
single_1.to_csv('csv/manual_clean.csv')

### Single_3

In [59]:
print(single_3.shape)
single_3.head()

(1, 10)


,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
1641,Northwest School Of The Arts,Public district,6-12,1083.0,14:1,Charlotte-Mecklenburg School District,9,Above average,North Carolina,3


In [60]:
single_3['Address'] = single_3['Address'].str.replace('North Carolina', '1415 Beatties Ford Rd, Charlotte, NC 28216')

In [61]:
single_3.head()

,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
1641,Northwest School Of The Arts,Public district,6-12,1083.0,14:1,Charlotte-Mecklenburg School District,9,Above average,"1415 Beatties Ford Rd, Charlotte, NC 28216",3


### Unrated_single

In [62]:
print(unrated_single.shape)
unrated_single.head(50)

(46, 10)


,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
881,"Bessie H Corhn Day Care Center 21924 Territorial Rd, Benton Harbor, MI, 49022",Private,PK,NaN,NaN,NaN,0,Currently unrated,"[Bessie H Corhn Day Care Center 21924 Territorial Rd, Benton Harbor, MI, 49022]",1
1002,"Kinder Care Learning Center #10872410 East Washington Street, Bloomington, IL, 61704",Private,PK,NaN,NaN,NaN,0,Currently unrated,"[Kinder Care Learning Center #10872410 East Washington Street, Bloomington, IL, 61704]",1
1384,"Farr James F71 Pearl St, Cambridge, MA, 02139",Private,7-12,NaN,NaN,NaN,0,Currently unrated,"[Farr James F71 Pearl St, Cambridge, MA, 02139]",1
1479,"The Goddard School - Carmel (City Center)160 Medical Drive, Carmel, IN, 46032",Private,PK-K,NaN,NaN,NaN,0,Currently unrated,"[The Goddard School - Carmel (City Center)160 Medical Drive, Carmel, IN, 46032]",1
1576,"Chesterbrook Academy Champaign UIUC2001 South Oak Street, Champaign, IL, 61820",Private,PK-K,NaN,NaN,NaN,0,Currently unrated,"[Chesterbrook Academy Champaign UIUC2001 South Oak Street, Champaign, IL, 61820]",1
1579,"Kinder Care Learning Center 1068410 Devonshire Drive, Champaign, IL, 61820",Private,PK-K,NaN,NaN,NaN,0,Currently unrated,"[Kinder Care Learning Center 1068410 Devonshire Drive, Champaign, IL, 61820]",1
1792,"3-6 Program (Clearfield HS)938 South 1000 East, Clearfield, UT, 84015",Public district,10-12,NaN,NaN,Davis District,0,"Currently unrated3-6 Program (Clearfield HS)938 South 1000 East, Clearfield, UT, 84015 Homes for sale","[3-6 Program (Clearfield HS)938 South 1000 East, Clearfield, UT, 84015]",1
1796,"College For Tots 1805 East 700 South, Clearfield, UT, 84015",Private,PK,NaN,NaN,NaN,0,Currently unrated,"[College For Tots 1805 East 700 South, Clearfield, UT, 84015]",1
1797,"Youth Health Associates Clearfield I II III (YIC)430 East 450 South, Clearfield, UT, 84015",Public district,6-12,NaN,NaN,Davis District,0,Currently unrated,"[Youth Health Associates Clearfield I II III (YIC)430 East 450 South, Clearfield, UT, 84015]",1
1995,"Kinder Care Learning Center #3005703625 Central Avenue, Columbus, IN, 47203",Private,PK-4,22.0,NaN,NaN,0,Currently unrated,"[Kinder Care Learning Center #3005703625 Central Avenue, Columbus, IN, 47203]",1


In [63]:
unrated_single[['Total students enrolled', 'Students per teacher', 'Students per teacher']] = unrated_single[['Total students enrolled', 'Students per teacher', 'Students per teacher']].fillna(0)

In [64]:
unrated_single.to_csv('csv/manual2.csv')

## Merge Data


In [65]:
df.append([unrated, single, miscellaneous, unrated_1, unrated_3, single_3], ignore_index=True)

,School,Type,Grades,Total students enrolled,Students per teacher,District,Score,Rating,Address,Length
0,Akron Early College High School,Public district,9-12,384.0,34:1,Akron City School District,10,Top rated,"225 South Main Street, Akron, OH, 44308",4
1,Revere Middle School,Public district,6-8,624.0,13:1,Revere Local School District,9,Above average,"3195 Spring Valley Road, Akron, OH, 44333",2
2,Arrowhead Primary Elementary School,Public district,K-4,345.0,20:1,Copley-Fairlawn City School District,8,Above average,"1600 Raleigh Boulevard, Akron, OH, 44321",2
3,Manchester Middle School,Public district,5-8,387.0,16:1,Manchester Local School District,8,Above average,"760 West Nimisila Road, Akron, OH, 44319",2
4,Nolley Elementary School,Public district,K-4,483.0,17:1,Manchester Local School District,8,Above average,"6285 Renninger Rd, Akron, OH, 44319",2
...,...,...,...,...,...,...,...,...,...,...
9542,Child Developmental Lab School,Private,PK,0.0,0,0,0,Currently unrated,"Gates Road, West Lafayette, IN, 47906",2
9543,Timber Ridge School,Private,6-12,66.0,0,0,0,Currently unrated,"Po Box 3160, Winchester, VA, 22604",2
9544,Heritage Christian Academy,Private,PK-8,89.0,0,0,0,Currently unrated,"Po Box 819, Winter Haven, FL, 33882",2
9545,Joy Quigley Day Care,Private,PK,0.0,0,NaN,0,Currently unrated,"94, Muscatine, IA, 52761",3


In [66]:
df.to_csv('schools.csv')